# AlphaFactory: a Quantitative Finance Research Plaftorm

Supervisor: DR. J.R. ZHANG 

Students: Liu Junyue, Sun Mengdie, Wang Xinlu, Yin Siyuan 

***


## Recap

### What we want to develop

* a quantitative investment research system 
* run fast, test strategies quickly

### What we have done
* Alpha research: SingleAssetResearch, MultiAssetResearch, AlphaManager
* Vectorization Backtesting


***

## Current status: last week

### Alpha research 
* Alpha manager functionality
* Alpha manager dashboard

### Backtesting
* Portfolio manager module
* visualization dashboard of strategy allocation

### Financial machine learning
* labelling
* Train weights, and sampling function

### Execution
* Live API combination (Futu api)


***

## Some demo

As mentioned last week, we would like to show some demo this week...

### Single asset alpha research

In [16]:
# first import some modules
import yfinance as yf
import pandas as pd
import inspect
from jupyter_dash import JupyterDash
import warnings
warnings.filterwarnings('ignore')

from alpha_research.SingleAssetResearch import SingleAssetResearch
from alpha_research.utils import print_code
from alpha_research.factor_zoo.alpha_101 import alpha_101, alpha_98

In [17]:
# get history data from api or local csv
tencent = yf.Ticker("0700.HK") # Tencent
data = tencent.history(period="7d", interval='1m')
data.index = data.index.tz_localize(None)
data.columns = [c.lower() for c in data.columns]
data.head()

,open,high,low,close,volume,dividends,stock splits
Datetime,,,,,,,
2020-07-13 09:30:00,541.0,544.0,540.5,541.0,0,0,0
2020-07-13 09:31:00,541.0,544.0,540.5,544.0,259653,0,0
2020-07-13 09:32:00,544.0,544.5,542.0,543.5,258268,0,0
2020-07-13 09:33:00,543.0,545.5,543.0,545.0,136753,0,0
2020-07-13 09:34:00,545.0,546.5,544.5,546.0,154189,0,0


In [18]:
alpha_res = SingleAssetResearch(data)

### design your own alpha

In this example we just use an alpha function from our own factor zoo

In [19]:
print_code(alpha_101)

def alpha_101(df: pd.DataFrame):
    """
    Alpha#101: ((close - open) / ((high - low) + .001))
    :param df:
    :return:
    """
    factor = (df['close'] - df['open']) / ((df['high'] - df['low']) + 0.001)
    return factor



In [20]:
# calculate the factor 
alpha_res.calculate_factor(alpha_101)

Datetime
2020-07-13 09:30:00    0.000000
2020-07-13 09:31:00    0.856898
2020-07-13 09:32:00   -0.199920
2020-07-13 09:33:00    0.799680
2020-07-13 09:34:00    0.499750
                         ...   
2020-07-14 15:31:00    0.000000
2020-07-14 15:32:00    0.000000
2020-07-14 15:33:00    0.000000
2020-07-14 15:34:00   -0.499500
2020-07-14 15:35:00    0.333111
Length: 637, dtype: float64

### Now is time to evaluate your alpha results.

We use jupyter dash to easy use dash in jupyter notebook enviornment

In [21]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
dash = JupyterDash(__name__, external_stylesheets=external_stylesheets)
app = alpha_res.get_evaluation_dash_app(dash)

In [22]:
app.run_server(mode='external', host='localhost', port=8055)

Dash app running on http://localhost:8055/


*** 

## More interactive demo: Multi Asset Research  

In [23]:
import pandas as pd
import numpy as np
import inspect
from jupyter_dash import JupyterDash


from alpha_research.MultiAssetResearch import MultiAssetResearch
from alpha_research.utils import print_code
from alpha_research.factor_zoo.alpha_101 import alpha_1
from alpha_research.factor_zoo.utils import *

In [24]:
# because it take some time to prepare the data, let's just use pre-prepared one.
# multi index. first index is date, second is code
data = pd.read_csv(r'../hsi_component.csv')
data['Date'] = pd.to_datetime(data['Date'])
data.set_index(['Date', 'code'], inplace=True)
data.head()

open    high     low   close    volume  dividends  \
Date       code                                                           
2010-06-17 0001.HK  43.094  43.094  42.639  43.046   3549306        0.0   
           0002.HK  37.355  37.388  36.592  36.824   2250739        0.0   
           0003.HK   5.796   5.859   5.764   5.840  14426271        0.0   
           0005.HK  46.139  46.232  45.705  45.984  10545908        0.0   
           0006.HK  25.578  25.744  25.384  25.440   3398904        0.0   

                    stock splits  
Date       code                   
2010-06-17 0001.HK           0.0  
           0002.HK           0.0  
           0003.HK           0.0  
           0005.HK           0.0  
           0006.HK           0.0

In [25]:
# to use sector analysis. Define sector of each code
sector = {'0001.HK': 'Commerce & Industry', '0002.HK': 'Utilities', '0003.HK': 'Utilities', '0005.HK': 'Finance',
         '0006.HK': 'Utilities', '0011.HK': 'Finance','0012.HK': 'Properties', '0016.HK': 'Properties', 
         '0017.HK': 'Properties','0019.HK': 'Commerce & Industry', '0066.HK': 'Commerce & Industry', 
         '0083.HK': 'Properties','0101.HK': 'Properties', '0151.HK': 'Commerce & Industry', 
         '0175.HK': 'Commerce & Industry','0267.HK': 'Commerce & Industry', '0386.HK': 'Commerce & Industry', 
         '0388.HK': 'Finance','0669.HK': 'Commerce & Industry', '0688.HK': 'Properties', 
         '0700.HK': 'Commerce & Industry','0762.HK': 'Commerce & Industry', '0823.HK': 'Properties', 
         '0857.HK': 'Commerce & Industry','0883.HK': 'Commerce & Industry', '0939.HK': 'Finance', 
         '0941.HK': 'Commerce & Industry','1038.HK': 'Utilities', '1044.HK': 'Commerce & Industry', 
         '1088.HK': 'Commerce & Industry','1093.HK': 'Commerce & Industry', '1109.HK': 'Properties', 
         '1177.HK': 'Commerce & Industry', '1398.HK': 'Finance', '1928.HK': 'Commerce & Industry', 
         '2007.HK': 'Properties','2018.HK': 'Commerce & Industry', '2313.HK': 'Commerce & Industry', 
         '2318.HK': 'Finance','2319.HK': 'Commerce & Industry', '2382.HK': 'Commerce & Industry', 
         '2388.HK': 'Finance','2628.HK': 'Finance', '3328.HK': 'Finance', '3988.HK': 'Finance', 
         '1299.HK': 'Finance','0027.HK': 'Commerce & Industry', '0288.HK': 'Commerce & Industry',
         '1113.HK': 'Properties', '1997.HK': 'Properties'}

In [26]:
# set a benchmark
benchmark = pd.read_csv(r'../^HSI_1986-12-31 00:00:00_2020-07-07 00:00:00.csv')
benchmark['Date'] = pd.to_datetime(benchmark['Date'])
benchmark.set_index('Date', inplace=True)
benchmark = benchmark['close']
benchmark.head()

Date
1986-12-31    2568.3
1987-01-02    2540.1
1987-01-05    2552.4
1987-01-06    2583.9
1987-01-07    2607.1
Name: close, dtype: float64

In [27]:
multi_study = MultiAssetResearch(data)
multi_study.set_asset_group(sector)
multi_study.set_benchmark(benchmark)

In [28]:
print_code(alpha_1)

def alpha_1(df: pd.DataFrame):
    """
    Alpha#1: (rank(Ts_ArgMax(SignedPower(((returns < 0) ? stddev(returns, 20) : close), 2.), 5)) - 0.5)
    :return:
    """
    _close = df['close']
    _ret = returns(_close)
    _cond = pd.Series(np.where(_ret < np.zeros_like(_ret.shape[0]), stddev(_ret, 20), _close), index=_close.index)
    return rank(ts_argmax(signedpower(_cond, 2), 5) - 0.5)



In [29]:
multi_study.calculate_factor(alpha_1)

In [30]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
dash = JupyterDash(__name__, external_stylesheets=external_stylesheets)
app = multi_study.get_evaluation_dash_app(dash)

In [31]:
app.run_server(host='localhost',mode='external', port=8060)

Dash app running on http://localhost:8060/


*** 
## Other last week work

### portfolio manager

Visualize the comparsion between strategies

![unsplash](img/pm1.png)

![unsplash](img/pm2.png)

*** 
## Next step

### Execution

* real-time moniter dash app

### Report Writing
